In [1]:
import pandas as pd
import numpy as np
import pyterrier_deepct
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

terrier-prf -SNAPSHOT jar not found, downloading to /mnt/ceph/storage/data-tmp/2021//angwr/.pyterrier...
Done


PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [ ]:
from xml.etree import ElementTree
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics2020.xml')
data = tree.getroot()
titles = []
title_ids = []
nums = []
for topic in data:
    titles.append(topic.find('title').text.lower()[:-1])
    num = topic.find('number').text
    nums.append(num)
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics.xml')
data = tree.getroot()    
for topic in data:
    titles.append(topic.find('title').text.lower()[:-1])
    num = topic.find('number').text
    nums.append(num)
topics = pd.DataFrame()
topics['qid'] = nums
topics['query'] = titles

qrels_path=("/workspace/src/evaluate_targers/qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

docnos = qrels.docno

In [11]:
import os
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            print(subdir)
            path = subdir + '/'+ file
            indexref = pt.IndexRef.of(path)
            bm25_rm3 = (pt.BatchRetrieve(indexref, wmodel="BM25") >> 
            pt.rewrite.RM3(indexref) >> 
            pt.BatchRetrieve(indexref, wmodel="BM25"))
            index = pt.IndexFactory.of(indexref)
            tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")
            bm25 = pt.BatchRetrieve(index, wmodel="BM25")
            dlm = pt.BatchRetrieve(index, wmodel="DirichletLM")
            dlm_rm3 = (pt.BatchRetrieve(indexref, wmodel="DirichletLM") >> 
            pt.rewrite.RM3(indexref) >> 
            pt.BatchRetrieve(indexref, wmodel="DirichletLM"))
            print(pt.Experiment([tf_idf, bm25, bm25_rm3, dlm, dlm_rm3], topics, qrels, eval_metrics=["ndcg_cut_5", "bpref", "ndcg_cut_25"]))
            

/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index
                                                name  ndcg_cut_5     bpref  \
0                                         BR(TF_IDF)    0.486396  0.724771   
1                                           BR(BM25)    0.482386  0.725052   
2  Compose(Compose(BR(BM25), <pyterrier.rewrite.R...    0.481536  0.749372   
3                                    BR(DirichletLM)    0.605145  0.726433   
4  Compose(Compose(BR(DirichletLM), <pyterrier.re...    0.525739  0.702217   

   ndcg_cut_25  
0     0.446201  
1     0.445090  
2     0.448463  
3     0.524329  
4     0.387855  
/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/pool_topicspd_index
                                                name  ndcg_cut_5     bpref  \
0                                         BR(TF_IDF)    0.493342  0.720428   
1                                           BR(BM

In [30]:
import os
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            print('\n' + subdir)
            for model in models:
                res= model.transform(topics)
                res_copy = res[res['docno'].isin(docnos)].copy()
                print(f'model : {model}, res: {pt.Utils.evaluate(res_copy,qrels,metrics=["ndcg_cut_5", "bpref", "ndcg_cut_25"])}')


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index
model : BR(TF_IDF), res: {'ndcg_cut_5': 0.642341683143697, 'bpref': 0.7204280566738166, 'ndcg_cut_25': 0.6857458417060763}
model : BR(BM25), res: {'ndcg_cut_5': 0.6393443031490417, 'bpref': 0.7190479895900244, 'ndcg_cut_25': 0.6821182756312493}
model : Compose(Compose(BR(BM25), <pyterrier.rewrite.RM3 object at 0x7f679893e890>), BR(BM25)), res: {'ndcg_cut_5': 0.6858085904068897, 'bpref': 0.7524243750930804, 'ndcg_cut_25': 0.7155137400853717}
model : BR(DirichletLM), res: {'ndcg_cut_5': 0.7063696387713557, 'bpref': 0.7181172056274511, 'ndcg_cut_25': 0.7200118026790665}
model : Compose(Compose(BR(DirichletLM), <pyterrier.rewrite.RM3 object at 0x7f680d860e10>), BR(DirichletLM)), res: {'ndcg_cut_5': 0.6711489415216131, 'bpref': 0.7001859279145797, 'ndcg_cut_25': 0.675224747160787}

/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/pool_to